In [168]:
import pandas as pd
import geopandas as gpd

In [169]:
df = pd.read_parquet("../Data/green_tripdata_2023-01.parquet")

In [170]:
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2023-01-01 00:26:10,2023-01-01 00:37:11,N,1.0,166,143,1.0,2.58,14.9,1.0,0.5,4.03,0.0,None,1.0,24.18,1.0,1.0,2.75
1,2,2023-01-01 00:51:03,2023-01-01 00:57:49,N,1.0,24,43,1.0,1.81,10.7,1.0,0.5,2.64,0.0,None,1.0,15.84,1.0,1.0,0.00
2,2,2023-01-01 00:35:12,2023-01-01 00:41:32,N,1.0,223,179,1.0,0.00,7.2,1.0,0.5,1.94,0.0,None,1.0,11.64,1.0,1.0,0.00
3,1,2023-01-01 00:13:14,2023-01-01 00:19:03,N,1.0,41,238,1.0,1.30,6.5,0.5,1.5,1.70,0.0,None,1.0,10.20,1.0,1.0,0.00
4,1,2023-01-01 00:33:04,2023-01-01 00:39:02,N,1.0,41,74,1.0,1.10,6.0,0.5,1.5,0.00,0.0,None,1.0,8.00,1.0,1.0,0.00


In [171]:
columns_to_keep = ["lpep_pickup_datetime", "lpep_dropoff_datetime", "PULocationID", "DOLocationID"]
df = df[columns_to_keep]

In [172]:
start_date = '2023-01-01'
end_date = '2023-01-31'

df = df[
    (df['lpep_pickup_datetime'] >= start_date) & (df['lpep_pickup_datetime'] <= end_date) &
    (df['lpep_dropoff_datetime'] >= start_date) & (df['lpep_dropoff_datetime'] <= end_date)
]

In [173]:
df['pickup_date'] = df['lpep_pickup_datetime'].dt.strftime('%Y-%m-%d')
df['dropoff_date'] = df['lpep_dropoff_datetime'].dt.strftime('%Y-%m-%d')

In [174]:
zones = pd.read_csv("../Data/taxi_zone_lookup.csv")

In [175]:
zones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       264 non-null    object
 2   Zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


In [176]:
zones[zones["LocationID"] == 57]

,LocationID,Borough,Zone,service_zone
56,57,Queens,Corona,Boro Zone


In [177]:
zones.isnull().sum()

LocationID      0
Borough         1
Zone            1
service_zone    2
dtype: int64

In [178]:
zones[zones.isnull().any(axis=1)]

,LocationID,Borough,Zone,service_zone
263,264,Unknown,NaN,NaN
264,265,NaN,Outside of NYC,NaN


In [179]:
zones.drop([263, 264], inplace=True)

In [180]:
df_filtered = df[~((df['PULocationID'].isin([264, 265])) | (df['DOLocationID'].isin([264, 265])))]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 65195 entries, 0 to 68081
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   lpep_pickup_datetime   65195 non-null  datetime64[us]
 1   lpep_dropoff_datetime  65195 non-null  datetime64[us]
 2   PULocationID           65195 non-null  int64         
 3   DOLocationID           65195 non-null  int64         
 4   pickup_date            65195 non-null  object        
 5   dropoff_date           65195 non-null  object        
dtypes: datetime64[us](2), int64(2), object(2)
memory usage: 3.5+ MB


In [181]:
df_filtered.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,pickup_date,dropoff_date
0,2023-01-01 00:26:10,2023-01-01 00:37:11,166,143,2023-01-01,2023-01-01
1,2023-01-01 00:51:03,2023-01-01 00:57:49,24,43,2023-01-01,2023-01-01
2,2023-01-01 00:35:12,2023-01-01 00:41:32,223,179,2023-01-01,2023-01-01
3,2023-01-01 00:13:14,2023-01-01 00:19:03,41,238,2023-01-01,2023-01-01
4,2023-01-01 00:33:04,2023-01-01 00:39:02,41,74,2023-01-01,2023-01-01


In [182]:
pickup = df_filtered.merge(zones[['LocationID', 'Zone']], left_on='PULocationID', right_on='LocationID', how='left')
dropoff = df_filtered.merge(zones[['LocationID', 'Zone']], left_on='DOLocationID', right_on='LocationID', how='left')
pickup.drop(columns=['LocationID'], inplace=True)
dropoff.drop(columns=['LocationID'], inplace=True)

In [183]:
pickup.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,pickup_date,dropoff_date,Zone
0,2023-01-01 00:26:10,2023-01-01 00:37:11,166,143,2023-01-01,2023-01-01,Morningside Heights
1,2023-01-01 00:51:03,2023-01-01 00:57:49,24,43,2023-01-01,2023-01-01,Bloomingdale
2,2023-01-01 00:35:12,2023-01-01 00:41:32,223,179,2023-01-01,2023-01-01,Steinway
3,2023-01-01 00:13:14,2023-01-01 00:19:03,41,238,2023-01-01,2023-01-01,Central Harlem
4,2023-01-01 00:33:04,2023-01-01 00:39:02,41,74,2023-01-01,2023-01-01,Central Harlem


In [184]:
pickup_counts = pickup[['pickup_date', 'PULocationID', 'Zone']].value_counts().reset_index()
dropoff_counts = dropoff[['dropoff_date', 'DOLocationID', 'Zone']].value_counts().reset_index()

In [185]:
pickup_counts.rename(columns={'PULocationID': 'LocationID', 'pickup_date' : 'date'}, inplace=True)
dropoff_counts.rename(columns={'DOLocationID': 'LocationID', 'dropoff_date' : 'date'}, inplace=True)

In [186]:
dropoff_counts

,date,LocationID,Zone,count
0,2023-01-06,74,East Harlem North,165
1,2023-01-23,75,East Harlem South,158
2,2023-01-03,75,East Harlem South,157
3,2023-01-17,75,East Harlem South,154
4,2023-01-12,75,East Harlem South,154
...,...,...,...,...
5662,2023-01-17,164,Midtown South,1
5663,2023-01-17,158,Meatpacking/West Village West,1
5664,2023-01-17,155,Marine Park/Mill Basin,1
5665,2023-01-17,149,Madison,1


In [187]:
pickup_counts.to_parquet('../Data/green_pickup_counts.parquet')
dropoff_counts.to_parquet('../Data/green_dropoff_counts.parquet')